Methods showcased in dev_pressure_curves facilitate fine-tuned and automated data transfer to dev_machine_learning 

In [1]:
#execute related notebooks within the scope of the current one to use their variables without the need for individual imports or file creation
#%run dev_pressure_curves.ipynb dev_machine_learning.ipynb

In [2]:
#Here, we import pressure curve files/object to run through a simple workflow. %store -[OPTION] var to -r : retrieve, -d : delete, -z : clear all  
%store -r pc 

In [3]:
methods_plot = pc.plot_methods()
methods_plot.show()

methods = pc.get_methods()
print("Methods: ", methods)

Methods:  ['SAA_411_Doc.M', 'SAA_411_5FU.M', 'SAA_411_Gem.M', 'SAA_411_Irino.M', 'SAA_411_Pac.M']


In [4]:
indices = pc.get_method_indices('SAA_411_Pac.M')
pac_plots = pc.plot_batches(indices)
pac_plots.show()

In [5]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative

processor = PressureCurvesMethodExtractFeaturesNative(period=10, bins=6, window_size=7)
pc = processor.run(pc)

In [6]:
batches = [batch for batch in pc.get_batches() if "Pac" in batch]
print("Batches: ", batches)

batches.sort()
print("Sorted by date: ", batches)

tests = batches[2:]

Batches:  ['210812_Pac 2021-08-12 10-30-07', '210906_Pac 2021-09-06 11-09-22', '210813_Pac 2021-08-13 10-37-27', '210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210910_Pac 2021-09-10 14-25-54']
Sorted by date:  ['210812_Pac 2021-08-12 10-30-07', '210813_Pac 2021-08-13 10-37-27', '210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [7]:
print("test batch dates: ", tests) # used to get train data, then discarded and test sets iteratively selected by increasing batch date

test batch dates:  ['210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [8]:
# create training set
from datetime import datetime, timedelta

first_test_group = tests[0]
print("First test group date: ", first_test_group)

first_test = first_test_group.split(" ")[-2:]
first_test = " ".join(first_test)
first_test = first_test.replace(":", "-")

date_threshold_min = first_test

First test group date:  210819_Pac 2021-08-19 15-50-49


In [9]:
train_indices = []
for i in indices:
    meta = pc.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = start_time.replace(":", "-")
    else:
        start_time = start_time.strftime("%Y-%m-%d %H-%M-%S")
    if batch_position > 5 and start_time < date_threshold_min:
        train_indices.append(i)

train_data = pc.get_features(train_indices)
train_metadata = pc.get_metadata(train_indices)
train_data.to_csv("dev/workflow_train_features.csv", index=False) # test data files will include batch date
train_metadata.to_csv("dev/workflow_train_metadata.csv", index=False)

train_size = len(train_indices)
print("Number of training curves: ", train_size)

Number of training curves:  18


In [10]:
fig_train=pc.plot_pressure_curves(train_indices)
fig_train.update_layout(showlegend=False)
fig_train.show()
#train_indices.extend([239, 245, 266, 117])# 117 is interesting

In [11]:
train_data.describe() # maybe this can be fed to model to guide feature selection for splits based on importance

,runtime,area_0.0_0.66,min_0.0_0.66,max_0.0_0.66,mean_0.0_0.66,std_0.0_0.66,range_0.0_0.66,residual_std_0.0_0.66,residual_noise_0.0_0.66,relative_change_0.0_0.66,...,min_3.33_3.975,max_3.33_3.975,mean_3.33_3.975,std_3.33_3.975,range_3.33_3.975,residual_std_3.33_3.975,residual_noise_3.33_3.975,relative_change_3.33_3.975,roc_3.33_3.975,abs_deviation_3.33_3.975
count,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,239.7,45.615564,59.168333,70.964444,69.082222,2.263282,11.796111,0.290950,32.101647,0.191190,...,40.111667,53.816667,46.444145,3.986602,13.705000,0.179988,25.286868,0.336882,0.077547,0.660728
std,0.0,0.138899,0.438718,0.245146,0.210181,0.097076,0.442641,0.028393,2.766722,0.009539,...,0.208616,0.273646,0.093785,0.017303,0.251027,0.021201,2.593377,0.007567,0.069328,0.089463
min,239.7,45.382050,58.140000,70.470000,68.727368,2.127925,11.170000,0.226699,24.606693,0.177064,...,39.760000,53.390000,46.339769,3.956200,13.200000,0.140243,19.756447,0.324318,-0.011761,0.546667
25%,239.7,45.535100,58.835000,70.840000,68.959511,2.195601,11.500000,0.275235,31.004150,0.184337,...,39.907500,53.630000,46.380019,3.972435,13.510000,0.166760,23.567972,0.333250,0.036038,0.578690
50%,239.7,45.597825,59.340000,70.940000,69.057180,2.238455,11.680000,0.298989,31.888993,0.190649,...,40.120000,53.745000,46.409308,3.983898,13.750000,0.180211,25.136645,0.334498,0.059937,0.660952
75%,239.7,45.703975,59.470000,71.020000,69.216090,2.329823,12.042500,0.307609,34.192084,0.196815,...,40.250000,53.977500,46.492596,4.003031,13.857500,0.193109,27.761196,0.342449,0.123459,0.710000
max,239.7,45.881100,59.780000,71.510000,69.484060,2.465887,12.720000,0.328231,36.184141,0.213450,...,40.430000,54.360000,46.677385,4.011977,14.110000,0.225321,29.002858,0.350559,0.219686,0.874286


In [12]:
# new tests sorted by date
tests = {}
for batch in batches:
    batch_date = batch.split(" ")[-2 : ]
    batch_date = " ".join(batch_date)

    if batch_date >= date_threshold_min: # search and collect test samples
        date_threshold_min = batch_date 
        
        test_indices = pc.get_batch_indices(batch)
        test_size = len(test_indices)

        while True:
            if test_size > train_size//3:
                test_indices_1 = test_indices[ : test_size//3]
                tests[f"{batch_date}_1"] = test_indices_1

                test_indices_2 = test_indices[test_size//3 : test_size//3 + len(test_indices_1)]
                tests[f"{batch_date}_2"] = test_indices_2
                
                test_indices_3 = test_indices[len(test_indices_1) + len(test_indices_2) : ]
                tests[f"{batch_date}_3"] = test_indices_3
                
                test_size = test_size//3

                if not test_size >= 3:
                    break
            else:
                break
    
    else:
        continue

In [13]:
print(sorted(tests)) # select test records

['2021-08-19 15-50-49_1', '2021-08-19 15-50-49_2', '2021-08-19 15-50-49_3', '2021-08-26 16-15-14_1', '2021-08-26 16-15-14_2', '2021-08-26 16-15-14_3', '2021-09-06 11-09-22_1', '2021-09-06 11-09-22_2', '2021-09-06 11-09-22_3', '2021-09-10 14-25-54_1', '2021-09-10 14-25-54_2', '2021-09-10 14-25-54_3']


In [14]:
date = '2021-08-19 15-50-49_1'

test_data = pc.get_features(tests[date])
test_metadata = pc.get_metadata(tests[date])

test_data.to_csv(f"dev/test_features_{date}.csv", index=False)
test_metadata.to_csv(f"dev/test_metadata_{date}.csv", index=False)
print(f"Test {date}: size: ", len(tests[date]), " indices:", tests[date])

fig_test=pc.plot_pressure_curves(indices = tests[date])
fig_test.update_layout(showlegend=False)
fig_test.write_image(f"dev/figures/fig_test_{date}_curves.png", width=550, height= 350, scale = 3)
fig_test.update_layout(showlegend=True)
fig_test.show()

fig_test_features = pc.plot_features(tests[date])
fig_test_features.update_layout(showlegend=False)
fig_test_features.write_image(f"dev/figures/fig_test_{date}_features.png", width=550, height= 350, scale = 3)
#fig_test_features.update_layout(showlegend=True)
#fig_test_features.show()

fig_test_features_raw = pc.plot_features_raw(tests[date])
fig_test_features_raw.update_layout(showlegend=False)
fig_test_features_raw.write_image(f"dev/figures/fig_test_{date}_features_raw.png", width=550, height= 350, scale = 3)
#fig_test_features_raw.update_layout(showlegend=True)
#fig_test_features_raw.show()

Test 2021-08-19 15-50-49_1: size:  5  indices: [116, 117, 118, 119, 120]


In [15]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

ml_ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ml_ana)

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html




MachineLearningAnalyses 
  variables: 18 rows, 67 columns
  metadata: 18 rows, 14 columns



In [16]:
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scaler = MachineLearningScaleFeaturesScalerSklearn(scaler_type = "StandardScaler")
ml_ana = scaler.run(ml_ana)

fig_train_features = ml_ana.plot_data()
fig_train_features.update_layout(title="Train set features")
fig_train_features.show()

In [17]:

from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iforest = MachineLearningMethodIsolationForestSklearn()

ml_ana = iforest.run(ml_ana)
ml_ana.train()
fig_train_scores = ml_ana.plot_scores()
fig_train_scores.show()

In [18]:
print(sorted(tests)) # select test records

['2021-08-19 15-50-49_1', '2021-08-19 15-50-49_2', '2021-08-19 15-50-49_3', '2021-08-26 16-15-14_1', '2021-08-26 16-15-14_2', '2021-08-26 16-15-14_3', '2021-09-06 11-09-22_1', '2021-09-06 11-09-22_2', '2021-09-06 11-09-22_3', '2021-09-10 14-25-54_1', '2021-09-10 14-25-54_2', '2021-09-10 14-25-54_3']


In [24]:
import os

date = '2021-08-19 15-50-49_1'
test_indices = tests[date]
test_data = pd.read_csv(f"dev/test_features_{date}.csv") if os.path.exists(f"dev/test_features_{date}.csv") else print(f"No files for this date: {date}")
test_metadata = pd.read_csv(f"dev/test_metadata_{date}.csv") if os.path.exists(f"dev/test_metadata_{date}.csv") else print(f"No files for this date: {date}")
ml_ana.predict(test_data, test_metadata)

#n_tests = 1
#outliers = ml_ana.test_prediction_outliers() # defaults: n_tests = 1, show_scores = False

#n_tests > 1
eval_results = ml_ana.test_prediction_outliers(n_tests = 20, show_scores = False)
results = eval_results["results"]

true_classes = results["true_classes"]
stability_score = results["stability_score"]
#true_classes.head()

print("Test set ", date, ": \n", test_metadata["index"].tolist()) 
print("\nModel stability: ", stability_score)

fig_test_curves = pc.plot_pressure_curves(test_metadata["index"].tolist())
fig_test_curves.show()

#fig_test_scores = ml_ana.plot_scores()
#fig_test_scores.write_image(f"dev/figures/fig_test_{date}_scores.png", width=1100, height= 350, scale = 3)
#fig_test_scores.update_layout(title=f"Test set {date} final run")
#fig_test_scores.show()

stability_plot = eval_results["stability_plot"]
stability_plot.show()

Some samples are unverified. Setting true_classes to majority class
Test set  2021-08-19 15-50-49_1 : 
 [116, 117, 118, 119, 120]

Model stability:  0.8286005269913502


In [ ]:
ml_ana.add_prediction()

# Done till here

In [75]:
# this plot can be migrated to native method of EvaluateModelStability
import plotly.graph_objects as go

test_record = pd.read_csv("dev/test_record.csv") if os.path.exists("dev/test_record.csv") else None
test_record = test_record.sort_values("date")

if test_record is not None:
    result_logs = []
    for date in test_record["date"]:
        result_logs.append(f"dev/test_{date}_classified_samples.csv") if os.path.exists(f"dev/test_{date}_classified_samples.csv") else print(f"No records for {date}")
    
    result_logs = [pd.read_csv(log) for log in result_logs]

    for log in result_logs:
        log["date"] = log["date"].astype(str).str[:19].str.replace("T", " ")

    result_logs = [log.to_string(index=False).replace("\n", "<br>") for log in result_logs]

    #test_record["date"] = pd.to_datetime(test_record["date"], format="%Y-%m-%dT%H-%M-%S-%f").dt.floor("s")
    #width = 600 
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=test_record["date"],
            y=test_record["threshold"],
            mode="lines+markers",
            name="Threshold",
            yaxis="y1",  
            hovertemplate=[
                "<br>Threshold: " + str(test_record["threshold"][i])  
                for i in range(len(test_record))
            ],
            line=dict(color="red", width=2, dash='dash'),
            marker=dict(size=8, symbol="circle")
        )
    )

    fig.add_trace(
        go.Bar(
            x=test_record["date"],
            y=test_record["outliers"],
            name="Outliers",
            yaxis="y2",
            width = 0.15, #width
            marker_color="blue",
            hovertext=result_logs,
            hoverinfo="text"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=test_record["date"],
            y=test_record["train_set"], 
            mode="lines+markers",
            name="Training curves",
            yaxis="y2",
            hovertemplate=[
                "<br>Training samples: " + str(test_record["train_set"][i]) +
                "<br>Test samples: " + str(test_record["test_set"][i]) +
                "<br>Outliers: " + str(test_record["outliers"][i]) +
                "<br>Outliers %: " + str(test_record["outliers_percent"][i]) 
                for i in range(len(test_record))
            ],
            line=dict(color="green", width=2, dash='solid'),
            marker=dict(size=8, symbol="star")
        )
    )

fig.update_layout(
    title="Detection accuracy over Test Runs and Training Set Size",
    xaxis=dict(
        tickvals=test_record["date"], 
        ticktext=[d[:19].replace("T", " ") for d in test_record["date"]], 
        tickangle=45,
        title="Test Dates"
    ),
    yaxis=dict(  
        title=dict(text="Threshold", font=dict(color="red")),
        tickfont=dict(color="red")
    ),
    yaxis2=dict(#train set and outlier realistic in scale. Uncomment y3 to adjust 
        title=dict(text="Outliers", font=dict(color="blue")),
        tickfont=dict(color="blue"),
        overlaying="y",
        side="right"
    ),
    bargap = 1, 
    template="simple_white",
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="top",
        orientation="h",  
        bgcolor="rgba(255,255,255,0.5)", 
        borderwidth=1  
    )
)
fig.write_image("dev/figures/fig_threshold_variation.png", width=1100, height= 350, scale= 3)
fig.show()


In [79]:
#%%timeit # ~1.9s +- ~30ms current setup
# EvaluateModelStability also prepares data for transfer from unsupervised to supervised learning by assigning true class labels
from src.StreamPort.machine_learning.methods import MachineLearningEvaluateModelStabilityNative

model_eval = MachineLearningEvaluateModelStabilityNative(test_records=summary)
results = model_eval.run()
true_classes = results["true_classes"] 
stability_score = results["stability_score"] 

print("Model stability score: ", stability_score)

confidence_plot = model_eval.plot_confidences()
confidence_plot.write_image("dev/figures/fig_model_stability.png", width=1100, height= 350, scale = 3)
confidence_plot.update_layout(showlegend=True)
confidence_plot.show()

Some samples are unverified. Setting true_classes to majority class
Model stability score:  0.7868348055262132


In [80]:
to_remove_from_tests = [116, 117, 121, 129, 132, 236, 272]
%store to_remove_from_tests

Stored 'to_remove_from_tests' (list)


In [81]:
# sorted. ensure identical column order 
new_data["label"] = true_classes["class_true"]
col = new_data.pop("label")
new_data.insert(14, "label", col)
print("New data: ", new_data.shape) 
new_data[metadata_columns].head()

New data:  (70, 31)


,index,name,path,batch,batch_position,idle_time,sample,method,timestamp,detector,pump,start_time,end_time,runtime,label
0,14,210812_Pac,C:/Users/Sandeep/Desktop/ExtractedSignals\2108...,210812_Pac 2021-08-12 10-30-07,1,142.0,Blank,SAA_411_Pac.M,2021-08-12 10:30:12,G7110B,G7110B,2021-08-12 10:32:20,2021-08-12 10:36:21,239.7,outlier
1,15,210812_Pac--002,C:/Users/Sandeep/Desktop/ExtractedSignals\2108...,210812_Pac 2021-08-12 10-30-07,2,39.0,Blank,SAA_411_Pac.M,2021-08-12 10:36:26,G1315C,G7110B,2021-08-12 10:37:00,2021-08-12 10:41:00,239.7,outlier
2,16,210812_Pac--003,C:/Users/Sandeep/Desktop/ExtractedSignals\2108...,210812_Pac 2021-08-12 10-30-07,3,39.0,Blank,SAA_411_Pac.M,2021-08-12 10:41:05,G1315C,G7110B,2021-08-12 10:41:39,2021-08-12 10:45:39,239.7,outlier
3,47,210813_Pac,C:/Users/Sandeep/Desktop/ExtractedSignals\2108...,210813_Pac 2021-08-13 10-37-27,1,5673.0,Blank,SAA_411_Pac.M,2021-08-13 10:37:33,G7110B,G7110B,2021-08-13 10:39:51,2021-08-13 10:43:51,239.7,outlier
4,48,210813_Pac--002,C:/Users/Sandeep/Desktop/ExtractedSignals\2108...,210813_Pac 2021-08-13 10-37-27,2,38.0,Blank,SAA_411_Pac.M,2021-08-13 10:43:56,G1315C,G7116B,2021-08-13 10:44:29,2021-08-13 10:48:30,239.7,normal


In [82]:
#combine all available labeled data, features and metadata included 
old_train_data = old_train_data.reset_index(drop=True)
new_data.reset_index(drop=True, inplace=True)

labeled_data = pd.concat([old_train_data, new_data], ignore_index=True)

# drop unclassified samples
labeled_data = labeled_data[labeled_data["label"] != "not set"]

metadata = labeled_data[metadata_columns]
labels = metadata["label"]
labeled_data = labeled_data.drop(columns= metadata_columns)
print("KNN Training Data: ", labeled_data.shape)

KNN Training Data:  (90, 16)


In [83]:
# create random splits of training and test sets. Shuffling and train-test-splitting with 'stratify' ensure a good distribution of classes so KNN learning is effective
from sklearn.model_selection import train_test_split
features_train, features_test, metadata_train, metadata_test = train_test_split(labeled_data, metadata, test_size=0.3, stratify=metadata["label"])

#%store features_train features_test metadata_train metadata_test

In [84]:

from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses
knn_ana = MachineLearningAnalyses(variables=features_train, metadata=metadata_train)

from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn
scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="StandardScaler")
knn_ana = scl.run(knn_ana)

from src.StreamPort.machine_learning.methods import MachineLearningMethodNearestNeighboursClassifierSklearn
knn = MachineLearningMethodNearestNeighboursClassifierSklearn(n_neighbors=1) #odd number for binary classification. 1 usually causes overfit, could be suitable for small amount of data
knn_ana = knn.run(knn_ana)

print(knn_ana)


NearestNeighboursAnalyses 
  variables: 63 rows, 16 columns
  metadata: 63 rows, 15 columns



In [85]:
knn_ana.plot_data()

In [86]:
knn_ana.train()
train_classes = knn_ana.get_training_labels()
print(train_classes)

    index    label
58    165   normal
13     54   normal
21     15  outlier
34    123   normal
5      23   normal
..    ...      ...
71    243   normal
25     49  outlier
44    150   normal
17     58   normal
28    117   normal

[63 rows x 2 columns]


In [87]:
train_labels = knn_ana.get_true_labels("train")
print("True labels: ",train_labels.shape)#, " ", train_labels)
print("Metadata labels: ", metadata_train[["index","label"]].shape)#, " ", metadata_train[["index","label"]])
if not train_labels.equals(train_classes):
    print("mismatch")
else:
    print("match")

True labels:  (63, 2)
Metadata labels:  (63, 2)
match


In [91]:
import umap
import numpy as np
import sklearn.preprocessing as scaler
import plotly.graph_objects as go

scaler = scaler.StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)

# UMAP for dimensionality reduction
umap_train_model = umap.UMAP(n_components=2)
features_test_reduced_umap = umap_train_model.fit_transform(features_train_scaled)

# Prepare the reduced data frame
reduced_data_umap = pd.DataFrame(features_test_reduced_umap, columns=['UMAP-1', 'UMAP-2'])
reduced_data_umap["predicted_labels"] = train_classes["label"]
reduced_data_umap["true_labels"] = train_labels["label"]
reduced_data_umap["index"] = train_labels["index"]

# Convert labels to binary values (0 for normal, 1 for outlier)
#reduced_data_umap["predicted_labels"] = np.where(reduced_data_umap["predicted_labels"] == "normal", 0, 1)
#reduced_data_umap["true_labels"] = np.where(reduced_data_umap["true_labels"] == "normal", 0, 1)

# Initialize the UMAP plot
fig_umap = go.Figure()

# Plot true labels (blue circles)
for i in range(len(reduced_data_umap)):
    if reduced_data_umap["true_labels"][i] == "outlier":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="blue",
                    size=10,
                    opacity=0.7,
                    symbol="circle",
                ),
                name="True labels",
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

    elif reduced_data_umap["true_labels"][i] == "normal":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="white",  # Color for normal labels
                    size=10,
                    opacity=0.4,
                    symbol="circle",
                ),
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

# Plot predicted labels (red 'x's)
for i in range(len(reduced_data_umap)):
    if reduced_data_umap["predicted_labels"][i] == "outlier":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="red",
                    size=8,
                    opacity=0.5,
                    symbol="x",
                ),
                name="Predicted Labels",
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

    elif reduced_data_umap["predicted_labels"][i] == "normal":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="white",  # Color for normal predicted labels
                    size=8,
                    opacity=0.4,
                    symbol="x",
                ),
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

# Update layout for the UMAP plot
fig_umap.update_layout(
    title="KNN Train Data Classification using UMAP reduction",
    xaxis_title="UMAP-1",
    yaxis_title="UMAP-2",
    template="plotly_dark",
    showlegend=False,
    hovermode="closest"
)

fig_umap.show()

In [88]:
knn_ana.predict(features_test, metadata_test)
classes = knn_ana.get_prediction_labels()
classes.reset_index(drop=True, inplace=True)
print("Predicted classes: ", classes.shape)#, " ", classes)

Predicted classes:  (27, 2)


In [89]:
true_labels = knn_ana.get_true_labels()
true_labels.reset_index(drop=True, inplace=True)
print("True classes: ", true_labels.shape)#, " ", true_labels)

True classes:  (27, 2)


In [90]:
# probabilities will sub for kNN where confidence was used for iForest 
probs = knn_ana.get_prediction_probabilities()
# print("Prediction probabilities: ", probs) # n_neighbours = 1 is not ideal but provides best results for small datasets. Use GridSearchCV (see dev_model_comparison) to find best params

In [ ]:
# from sklearn.manifold import TSNE


# n_samples = len(features_test_scaled)
# tsne = TSNE(
#     perplexity=n_samples - 1 if 30 > n_samples else 30,
#     n_components=2, 
#     random_state=42
#     )
# features_test_reduced = tsne.fit_transform(features_test_scaled)

# print(f"Scaled Data NaNs: {pd.isna(features_test_scaled).sum()}")

# reduced_data = pd.DataFrame(features_test_reduced, columns=['TSNE-1', 'TSNE-2'])

# reduced_data["predicted_labels"] = classes["label"]
# reduced_data["true_labels"] = true_labels["label"]
# reduced_data["index"] = true_labels["index"]
# print(reduced_data)


# reduced_data["predicted_labels"] = np.where(reduced_data["predicted_labels"] == "normal", 0, 1)
# reduced_data["true_labels"] = np.where(reduced_data["true_labels"] == "normal", 0, 1)
# print(f"Classes: {classes.shape}")
# print(f"True labels: {true_labels.shape}")
# print(f"Reduced data: {reduced_data.shape}")

# fig = go.Figure()

# for i in range(len(reduced_data)):
#     if reduced_data["true_labels"][i] == 1:
#         fig.add_trace(
#             go.Scatter(
#                 x=[reduced_data["TSNE-1"][i]],  
#                 y=[reduced_data["TSNE-2"][i]],  
#                 mode="markers",
#                 marker=dict(
#                     color="blue",  
#                     size=10,
#                     opacity=0.7,
#                     symbol="circle",
#                 ),
#                 name="True labels",
#                 hovertext=f"Index: {reduced_data['index'][i]}<br>Predicted label: {reduced_data['predicted_labels'][i]}<br>True label: {reduced_data['true_labels'][i]}"
#             )
#         )

#     if reduced_data["predicted_labels"][i] == 1:
#         fig.add_trace(
#             go.Scatter(
#                 x=[reduced_data["TSNE-1"][i]], 
#                 y=[reduced_data["TSNE-2"][i]], 
#                 mode="markers",
#                 marker=dict(
#                     color="red",  
#                     size=8,
#                     opacity=0.5,
#                     symbol="x",
#                 ),
#                 name="Predicted Labels",
#                 hovertext=f"Index: {reduced_data['index'][i]}<br>Predicted label: {reduced_data['predicted_labels'][i]}<br>True label: {reduced_data['true_labels'][i]}"
#             )
#         )

#     else:
#          fig.add_trace(
#             go.Scatter(
#                 x=[reduced_data["TSNE-1"][i]],  # Only plot the current row
#                 y=[reduced_data["TSNE-2"][i]],  # Only plot the current row
#                 mode="markers",
#                 marker=dict(
#                     color="white",  # White color for true labels with value 0
#                     size=10,
#                     opacity=0.4,
#                     symbol="circle",
#                 ),
#                 hovertext=f"Index: {reduced_data['index'][i]}<br>Predicted label: {reduced_data['predicted_labels'][i]}<br>True label: {reduced_data['true_labels'][i]}"
#             )
#         )        

# fig.update_layout(
#     title = "KNN classification results using t-SNE reduction",
#     xaxis_title = "TSNE-1",
#     yaxis_title = "TSNE-2",
#     template = "plotly_dark",
#     showlegend = False,
#     hovermode = "closest"
# )

# fig.show()

In [ ]:
# UMAP for dimensionality reduction while maintaining local relationships
umap_test_model = umap.UMAP(n_components=2)
features_test_scaled = scaler.fit_transform(features_test)
features_test_reduced_umap = umap_test_model.fit_transform(features_test_scaled)

# Prepare the reduced data frame
reduced_data_umap = pd.DataFrame(features_test_reduced_umap, columns=['UMAP-1', 'UMAP-2'])
reduced_data_umap["predicted_labels"] = classes["label"]
reduced_data_umap["true_labels"] = true_labels["label"]
reduced_data_umap["index"] = true_labels["index"]

# Convert labels to binary values (0 for normal, 1 for outlier)
#reduced_data_umap["predicted_labels"] = np.where(reduced_data_umap["predicted_labels"] == "normal", 0, 1)
#reduced_data_umap["true_labels"] = np.where(reduced_data_umap["true_labels"] == "normal", 0, 1)

# Initialize the UMAP plot
fig_umap = go.Figure()

# Plot true labels (blue circles)
for i in range(len(reduced_data_umap)):
    if reduced_data_umap["true_labels"][i] == "outlier":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="blue",
                    size=10,
                    opacity=0.7,
                    symbol="circle",
                ),
                name="True labels",
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

    elif reduced_data_umap["true_labels"][i] == "normal":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="white",  # Color for normal labels
                    size=10,
                    opacity=0.4,
                    symbol="circle",
                ),
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

# Plot predicted labels (red 'x's)
for i in range(len(reduced_data_umap)):
    if reduced_data_umap["predicted_labels"][i] == "outlier":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="red",
                    size=8,
                    opacity=0.5,
                    symbol="x",
                ),
                name="Predicted Labels",
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

    elif reduced_data_umap["predicted_labels"][i] == "normal":
        fig_umap.add_trace(
            go.Scatter(
                x=[reduced_data_umap["UMAP-1"][i]],
                y=[reduced_data_umap["UMAP-2"][i]],
                mode="markers",
                marker=dict(
                    color="white",  # Color for normal predicted labels
                    size=8,
                    opacity=0.4,
                    symbol="x",
                ),
                hovertext=f"Index: {reduced_data_umap['index'][i]}<br>Predicted label: {reduced_data_umap['predicted_labels'][i]}<br>True label: {reduced_data_umap['true_labels'][i]}"
            )
        )

# Update layout for the UMAP plot
fig_umap.update_layout(
    title="KNN Test Classification Results using UMAP reduction",
    xaxis_title="UMAP-1",
    yaxis_title="UMAP-2",
    template="plotly_dark",
    showlegend=False,
    hovermode="closest"
)

fig_umap.show()
#fig_umap.write_image("dev/figures/fig_knn_test_predictions_umap.png", width=1100, height= 350, scale = 3)

In [ ]:
from sklearn.decomposition import PCA

# PCA for dimensionality reduction
pca_model = PCA(n_components=2)
features_test_reduced_pca = pca_model.fit_transform(features_test_scaled)

covariance = pca_model.get_covariance()
# Prepare the reduced data frame for PCA plot
reduced_data_pca = pd.DataFrame(features_test_reduced_pca, columns=['PCA-1', 'PCA-2'])
reduced_data_pca["predicted_labels"] = classes["label"]
reduced_data_pca["true_labels"] = true_labels["label"]
reduced_data_pca["index"] = true_labels["index"]

# Initialize the PCA plot
fig_pca = go.Figure()

for i in range(len(reduced_data_pca)):
    if reduced_data_pca["true_labels"][i] == "outlier":
        fig_pca.add_trace(
            go.Scatter(
                x=[reduced_data_pca["PCA-1"][i]],
                y=[reduced_data_pca["PCA-2"][i]],
                mode="markers",
                marker=dict(
                    color="blue",
                    size=10,
                    opacity=0.8,
                    symbol="circle",
                ),
                name="True labels",
                hovertext=f"Index: {reduced_data_pca['index'][i]}<br>Predicted label: {reduced_data_pca['predicted_labels'][i]}<br>True label: {reduced_data_pca['true_labels'][i]}"
            )
        )

    if reduced_data_pca["predicted_labels"][i] == "outlier":
        fig_pca.add_trace(
            go.Scatter(
                x=[reduced_data_pca["PCA-1"][i]],
                y=[reduced_data_pca["PCA-2"][i]],
                mode="markers",
                marker=dict(
                    color="red",
                    size=8,
                    opacity=0.5,
                    symbol="x",
                ),
                name="Predicted Labels",
                hovertext=f"Index: {reduced_data_pca['index'][i]}<br>Predicted label: {reduced_data_pca['predicted_labels'][i]}<br>True label: {reduced_data_pca['true_labels'][i]}"
            )
        )
    else:
        fig_pca.add_trace(
            go.Scatter(
                x=[reduced_data_pca["PCA-1"][i]],
                y=[reduced_data_pca["PCA-2"][i]],
                mode="markers",
                marker=dict(
                    color="white",  # White color for true labels with value 0
                    size=10,
                    opacity=0.4,
                    symbol="circle",
                ),
                hovertext=f"Index: {reduced_data_pca['index'][i]}<br>Predicted label: {reduced_data_pca['predicted_labels'][i]}<br>True label: {reduced_data_pca['true_labels'][i]}"
            )
        )

# Update the layout
fig_pca.update_layout(
    title="KNN classification results using PCA reduction",
    xaxis_title="PCA-1",
    yaxis_title="PCA-2",
    template="plotly_dark",
    showlegend=False,
    hovermode="closest"
)

fig_pca.show()
#fig_pca.write_image("dev/figures/fig_knn_test_predictions_pca.png", width=1100, height= 350, scale = 3)

In [ ]:
#print("covariance:", covariance)
covariance_matrix = []
for i in range(len(covariance)):
    covariance_matrix.append(pd.DataFrame(covariance[i], columns=[features_test.columns[i]], index =features_test.columns ))

covariance_matrix = pd.concat(covariance_matrix, axis = 1)

fig = go.Figure(data=go.Heatmap(
    z=covariance_matrix.values,
    x=covariance_matrix.columns,
    y=covariance_matrix.index,
    colorscale='Viridis',
    colorbar=dict(title='Covariance'),
    zmin=np.min(covariance_matrix.values),
    zmax=np.max(covariance_matrix.values)
))

fig.update_layout(
    title='PCA Covariance Matrix Heatmap',
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    width=700,
    height=700
)

fig.show()